In [46]:
import numpy as np
import pandas as pd
import math
from typing import Tuple, List
import re
import os

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [47]:
#Reading file

def parse_xy_data(filepath, delimiter=None):
    """
    Reads a CSV-like file with x, y data and returns a list of (x, y) tuples.

    This function is designed to handle different delimiters (like ';' or ' ')
    and assumes that commas (',') are used as decimal separators, based on
    the provided image.

    Args:
        filepath (str): The path to the data file.
        delimiter (str, optional): The column delimiter (e.g., ';', ' '). 
                                   If None, the function will try to 
                                   auto-detect it.

    Returns:
        list: A list of (x, y) float tuples.
              Returns an empty list if the file cannot be read or is empty.
    """
    data_points = []
    detected_delimiter = delimiter
    
    # --- 1. Delimiter Sniffing (if not provided) ---
    if detected_delimiter is None:
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                # Read the first non-empty line to guess
                first_line = ""
                for line in f:
                    first_line = line.strip()
                    if first_line:
                        break
                
                if ';' in first_line:
                    detected_delimiter = ';'
                elif ' ' in first_line:
                    # Check if it's likely a space delimiter
                    parts = re.split(r'\s+', first_line)
                    if len(parts) == 2:
                        try:
                            # Try to parse to see if it makes sense
                            float(parts[0].replace(',', '.'))
                            float(parts[1].replace(',', '.'))
                            detected_delimiter = ' '
                        except (ValueError, IndexError):
                             # Not a valid 2-column space-delimited float line
                             pass
                
                if detected_delimiter is None and ',' in first_line:
                    # Comma is the last guess, as it's ambiguous with decimal
                    detected_delimiter = ','
                
                if detected_delimiter is None:
                    # Final fallback based on your image
                    print("Warning: Could not auto-detect delimiter. Falling back to ';'.")
                    detected_delimiter = ';'
        except Exception as e:
            print(f"Error opening/reading file for sniffing: {e}")
            return [] # Return empty list on error
    
    print(f"Using delimiter: '{detected_delimiter}'")

    # --- 2. File Parsing ---
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line_number, line in enumerate(f, 1):
                line = line.strip()
                if not line or line.startswith('#'):
                    continue # Skip empty lines or comment lines

                # Split the line by the detected delimiter
                if detected_delimiter == ' ':
                    # Use regex split for spaces to handle multiple spaces
                    parts = re.split(r'\s+', line)
                else:
                    parts = line.split(detected_delimiter)

                # Ensure we have exactly two columns
                if len(parts) == 2:
                    x_str, y_str = parts
                    
                    try:
                        # KEY STEP: Replace comma with dot for float conversion
                        x_val = float(x_str.strip().replace(',', '.'))
                        y_val = float(y_str.strip().replace(',', '.'))
                        data_points.append((x_val, y_val))
                    except ValueError as e:
                        # Warn if conversion to float fails
                        print(f"Warning: Could not parse numbers on line {line_number}: '{line}'. Error: {e}")
                else:
                    # Warn if the line doesn't have exactly two parts
                    print(f"Warning: Skipping malformed line {line_number}: '{line}'. Expected 2 columns, found {len(parts)}")
    
    except FileNotFoundError:
        print(f"Error: File not found at '{filepath}'")
        return []
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return []

    return data_points

In [48]:
print("--- Example: Reading '19data.csv' ---")
    
# You would replace '19data.csv' with the path to your actual file
file_to_read = '20251GKtest11.csv' 
    
# Check if the file exists before trying to read it
if os.path.exists(file_to_read):
    # Call the function to parse the file.
    # It will try to auto-detect the delimiter.
    all_points = parse_xy_data(file_to_read)
        
    if all_points:
        print(f"Successfully parsed {len(all_points)} data points:")
        df_input = pd.DataFrame(all_points, columns=['x', 'y'])
        print(df_input.to_string(index=False))
    else:
        print(f"Could not parse any data points from '{file_to_read}'.")
        print("Please check the file format and warnings above.")
            
else:
    print(f"Error: The file '{file_to_read}' was not found.")
    print("Please create this file or change 'file_to_read' variable")
    print("to point to your existing data file.")

--- Example: Reading '19data.csv' ---
Using delimiter: ';'
Successfully parsed 104 data points:
     x       y
 1.000 -5.1991
 1.115 -4.3239
 1.230 -3.7284
 1.345 -3.2661
 1.460 -2.9082
 1.575 -2.6225
 1.690 -2.3954
 1.805 -2.2098
 1.920 -2.0282
 2.035 -1.8916
 2.150 -1.7967
 2.265 -1.6635
 2.380 -1.5819
 2.495 -1.5127
 2.610 -1.3977
 2.725 -1.3814
 2.840 -1.2792
 2.955 -1.2786
 3.070 -1.2207
 3.185 -1.1791
 3.300 -1.0930
 3.415 -1.0193
 3.530 -1.0264
 3.645 -0.9742
 3.760 -0.9384
 3.875 -0.8979
 3.990 -0.8110
 4.105 -0.8056
 4.220 -0.7548
 4.335 -0.7416
 4.450 -0.7049
 4.565 -0.6784
 4.680 -0.6445
 4.795 -0.6134
 4.910 -0.5789
 5.025 -0.5578
 5.140 -0.5278
 5.255 -0.4917
 5.370 -0.5137
 5.485 -0.4717
 5.600 -0.4697
 5.715 -0.4589
 5.830 -0.4215
 5.945 -0.4250
 6.060 -0.3984
 6.175 -0.3300
 6.290 -0.3724
 6.405 -0.3815
 6.520 -0.3639
 6.635 -0.3482
 6.750 -0.3021
 6.865 -0.2574
 6.980 -0.3142
 7.095 -0.2815
 7.210 -0.2804
 7.325 -0.2677
 7.440 -0.2675
 7.555 -0.2496
 7.670 -0.2640
 7.7

# Loop Inverse

## Forward Algorithm

* Conditions
    * The data points $(x_i, y_i)$ must be **evenly spaced** (constant $h$).
    * The function $f(x)$ must be **monotonic** on the chosen interpolation interval.

* Input:
    * A set of $n+1$ evenly spaced, monotonic points $P = [(x_0, y_0), \dots, (x_n, y_n)]$.
    * A target value $\bar{y}$.
    * A tolerance $\epsilon$.

1. Output:
An approximated value $\bar{x}$ such that $f(\bar{x}) \approx \bar{y}$.

* Setup and Difference Table

    1.  **Select Interval:** Choose a monotonic, evenly-spaced subset of points from the data. Let the starting point be $(x_0, y_0)$ and the step size be $h$.
    2.  **Difference Table:** Compute the finite difference table for the selected points using `EvenDifference(points, condition=1)`.
    3.  **Get Coefficients:** Extract the forward difference coefficients (the top diagonal) $C_i = \Delta^i y_0$ for $i = 0, \dots, n$.

2. Formulate the Fixed-Point Problem

    1.  **Start with Newton's Formula:** We start with the equation $P(t) = \bar{y}$, where $t = (x - x_0) / h$:
    $\bar{y} = C_0 + \frac{C_1}{1!}t + \frac{C_2}{2!}t(t-1) + \frac{C_3}{3!}t(t-1)(t-2) + \dots$
    (Note: $C_0 = y_0$, $C_1 = \Delta y_{1}$, $C_2 = \Delta^2 y_{2}$, etc.)

    2.  **Isolate $t$:** Rearrange the equation to solve for the $t$ in the first-order term:
    $C_1 t = (\bar{y} - y_0) - \left[ \frac{C_2}{2!}t(t-1) + \frac{C_3}{3!}t(t-1)(t-2) + \dots \right]$

    3.  **Define $\varphi(t)$:** This gives the iteration function $t = \varphi(t)$:
    $\varphi(t) = \frac{1}{C_1} \left( (\bar{y} - y_0) - \sum_{i=2}^{n} \left[ \frac{C_i}{i!} \prod_{j=0}^{i-1}(t-j) \right] \right)$

3. Perform the Iteration

    1.  **Initial Guess $t_0$:** A good initial guess is found by ignoring all terms where $i \ge 2$:
    $t_0 = \frac{\bar{y} - y_0}{C_1}$

    2.  **Iterate:** Repeatedly calculate $t_{k+1} = \varphi(t_k)$.

    3.  **Stopping Condition:** Stop the iteration when the error is within tolerance:
    $|t_{k+1} - t_k| < \epsilon$

4. Find Final Solution

    1.  **Get $t$:** The final converged value is $t_{\text{final}}$.
    2.  **Convert to $x$:** Convert $t_{\text{final}}$ back to $x$ using the original transformation:
    $\bar{x} = x_0 + t_{\text{final}} \cdot h$

## Backward Algorithm

* Conditions
    * The data points $(x_i, y_i)$ must be **evenly spaced** (constant $h$).
    * The function $f(x)$ must be **monotonic** on the chosen interpolation interval.
    * This method is ideal when the target $\bar{y}$ is closer to the *end* of the interval ($y_n$) than the beginning ($y_0$).

* Input:
    * A set of $n+1$ evenly spaced, monotonic points $P = [(x_0, y_0), \dots, (x_n, y_n)]$.
    * A target value $\bar{y}$.
    * A tolerance $\epsilon$.

* Output:
An approximated value $\bar{x}$ such that $f(\bar{x}) \approx \bar{y}$.

1. Setup and Difference Table

    1.  **Select Interval:** Choose a monotonic, evenly-spaced subset of points. Let the *end* point be $(x_n, y_n)$ and the step size be $h$.
    2.  **Difference Table:** Compute the finite difference table for the selected points using `EvenDifference(points, condition=0)`.
    3.  **Get Coefficients:** Extract the **backward** difference coefficients (the bottom diagonal) $C_i = \Delta^i y_{n-i}$ for $i = 0, \dots, n$.

2. Formulate the Fixed-Point Problem

    1.  **Start with Newton's Backward Formula:** We start with $P(t) = \bar{y}$, where $t = (x - x_n) / h$.
    $\bar{y} = C_0 + \frac{C_1}{1!}t + \frac{C_2}{2!}t(t+1) + \frac{C_3}{3!}t(t+1)(t+2) + \dots$
    (Note: $C_0 = y_n$, $C_1 = \Delta y_{n-1}$, $C_2 = \Delta^2 y_{n-2}$, etc.)

    2.  **Isolate $t$:** Rearrange the equation to solve for the $t$ in the first-order term [cite: image_7fa3a1.png].
    $C_1 t = (\bar{y} - y_n) - \left[ \frac{C_2}{2!}t(t+1) + \frac{C_3}{3!}t(t+1)(t+2) + \dots \right]$

    3.  **Define $\varphi(t)$:** This gives the iteration function $t = \varphi(t)$:
    $\varphi(t) = \frac{1}{C_1} \left( (\bar{y} - y_n) - \sum_{i=2}^{n} \left[ \frac{C_i}{i!} \prod_{j=0}^{i-1}(t+j) \right] \right)$

3. Perform the Iteration

    1.  **Initial Guess $t_0$:** A good initial guess is found by ignoring all terms where $i \ge 2$ [cite: image_7fa3a1.png]:
    $t_0 = \frac{\bar{y} - y_n}{C_1}$

    2.  **Iterate:** Repeatedly calculate $t_{k+1} = \varphi(t_k)$ [cite: k1m1nonawa4869/fami304x-numerical-analysis/FaMI304x-Numerical-Analysis-main/Week2-Roots-Function/Part1-Root-Isolation/pp5_fixed_point_iteration.ipynb].

    3.  **Stopping Condition:** Stop the iteration when the error is within tolerance:
    $|t_{k+1} - t_k| < \epsilon$

4. Find Final Solution

    1.  **Get $t$:** The final converged value is $t_{\text{final}}$.
    2.  **Convert to $x$:** Convert $t_{\text{final}}$ back to $x$ using the backward transformation:
    $\bar{x} = x_n + t_{\text{final}} \cdot h$

## Monotonic Interval

In [49]:
def find_monotonic_intervals(points: List[Tuple[float, float]]) -> pd.DataFrame:
    """
    Analyzes a list of (x, y) points and finds all continuous monotonic
    (non-decreasing or non-increasing) intervals.
    """
    if len(points) < 2:
        return pd.DataFrame(columns=["Type", "Start (x, y)", "End (x, y)"])
    intervals = []
    start_point = points[0]
    current_direction = np.sign(points[1][1] - points[0][1])
    if current_direction == 0:
        current_direction = 1 
    for i in range(1, len(points) - 1):
        x_i, y_i = points[i]
        x_ip1, y_ip1 = points[i+1]
        new_direction = np.sign(y_ip1 - y_i)
        if new_direction != 0 and new_direction != current_direction:
            interval_type = "Increasing" if current_direction > 0 else "Decreasing"
            intervals.append((interval_type, start_point, points[i]))
            start_point = points[i]
            current_direction = new_direction
    interval_type = "Increasing" if current_direction > 0 else "Decreasing"
    intervals.append((interval_type, start_point, points[-1]))
    df = pd.DataFrame(intervals, columns=["Type", "Start Point (x, y)", "End Point (x, y)"])
    return df

In [50]:
print("\n--- Finding Monotonic Intervals ---")
intervals_df = find_monotonic_intervals(all_points)
print(intervals_df.to_string(index=False))


--- Finding Monotonic Intervals ---
      Type Start Point (x, y)  End Point (x, y)
Increasing     (1.0, -5.1991)  (3.415, -1.0193)
Decreasing   (3.415, -1.0193)   (3.53, -1.0264)
Increasing    (3.53, -1.0264)  (5.255, -0.4917)
Decreasing   (5.255, -0.4917)   (5.37, -0.5137)
Increasing    (5.37, -0.5137)   (5.83, -0.4215)
Decreasing    (5.83, -0.4215)   (5.945, -0.425)
Increasing    (5.945, -0.425)    (6.175, -0.33)
Decreasing     (6.175, -0.33)  (6.405, -0.3815)
Increasing   (6.405, -0.3815)  (6.865, -0.2574)
Decreasing   (6.865, -0.2574)   (6.98, -0.3142)
Increasing    (6.98, -0.3142)  (7.555, -0.2496)
Decreasing   (7.555, -0.2496)    (7.67, -0.264)
Increasing     (7.67, -0.264)   (7.785, -0.254)
Decreasing    (7.785, -0.254)  (8.015, -0.2585)
Increasing   (8.015, -0.2585)   (8.13, -0.2511)
Decreasing    (8.13, -0.2511)  (8.245, -0.2669)
Increasing   (8.245, -0.2669)    (8.59, -0.214)
Decreasing     (8.59, -0.214)  (8.705, -0.2618)
Increasing   (8.705, -0.2618)  (8.935, -0.1932)
Dec

## Get a subset range

In [51]:
def get_points_in_range(points: List[Tuple[float, float]], 
                        x_start: float, 
                        x_end: float) -> List[Tuple[float, float]]:
    """
    Filters a list of (x, y) tuples to return only those within a 
    specified x-range [x_start, x_end].
    """
    subset_points = [
        (x, y) for (x, y) in points 
        if x_start <= x <= x_end
    ]
    return subset_points


In [52]:
# Select points for the second interval [9.7, 10.0]
# This interval is suitable for Newton's FORWARD method

x_start_demo = 2.265
x_end_demo = 2.610
    
monotonic_points_fwd = get_points_in_range(all_points, x_start_demo, x_end_demo)
    
print(f"Subset of points from x={x_start_demo} to x={x_end_demo}:")
df_subset = pd.DataFrame(monotonic_points_fwd, columns=['x (swap)', 'y (swap)'])
print(df_subset.to_string(index=False))

Subset of points from x=2.265 to x=2.61:
 x (swap)  y (swap)
    2.265   -1.6635
    2.380   -1.5819
    2.495   -1.5127
    2.610   -1.3977


## Even Difference Table

In [53]:
def EvenDifference(points, condition):
    """
    Calculates the finite difference table and extracts coefficients.
    """
    x_values = [p[0] for p in points]
    y_values = [p[1] for p in points]
    n = len(y_values)

    # Internal calculation table
    diff_calc_table = np.full((n, n), np.nan)
    diff_calc_table[:, 0] = y_values
    for j in range(1, n):
        for i in range(n - j):
            diff_calc_table[i, j] = diff_calc_table[i+1, j-1] - diff_calc_table[i, j-1]

    # Format the output DataFrame
    data = {'x_i': x_values, 'y_i': y_values}
    for j in range(1, n):
        col_name = f'Order {j}'
        col_data = np.full(n, np.nan)
        col_data[:(n-j)] = diff_calc_table[:(n-j), j]
        data[col_name] = col_data
    
    df = pd.DataFrame(data)

    # Extract coefficients
    if condition == 1: # Newton-Gregory Forward (top diagonal)
        coefficients = diff_calc_table[0, :].tolist()
    else: # Newton-Gregory Backward (bottom diagonal)
        coeffs = []
        for j in range(n):
            coeffs.append(diff_calc_table[n-1-j, j])
        coefficients = coeffs

    return df, coefficients


## Core Function

In [54]:
def evaluate_basis_product(t, i, condition):
    if condition == 1:
        """ Helper function to calculate t(t-1)...(t-(i-1)) """
        if i == 0:
            return 1.0
        product = 1.0
        for j in range(i):
            product *= (t - j)
        return product
    
    if condition == 0:
        """ Helper function to calculate t(t+1)...(t+(i-1)) """
        if i == 0:
            return 1.0
        product = 1.0
        for j in range(i):
            product *= (t + j)
        return product

In [55]:
def phi_function(t, y_target, y_tmp, C_coeffs, condition):
    if condition == 1:
        """
        Calculates phi(t) based on the formula derived from Newton's Forward polynomial.
        phi(t) = (1/C_1) * [ (y_target - y_0) - sum_{i=2 to n}( C_i/i! * B_i(t) ) ]
        where C_i = Delta^i y_0
        """
        n = len(C_coeffs) - 1
        C_1 = C_coeffs[1]
    
        y_diff = y_target - y_tmp #y_tmp = y_0
    
        sum_higher_order = 0.0
        for i in range(2, n + 1):
            C_i = C_coeffs[i]
            D_i = C_i / math.factorial(i)
            B_i_t = evaluate_basis_product(t, i, condition)
            sum_higher_order += D_i * B_i_t

        return (y_diff - sum_higher_order) / C_1

    if condition == 0:
        """
        Calculates phi(t) based on the formula derived from Newton's Backward polynomial.
        phi(t) = (1/C_1) * [ (y_target - y_n) - sum_{i=2 to n}( C_i/i! * B_i(t) ) ]
        where C_i = Delta^i y_{n-i}
        """
        n = len(C_coeffs) - 1
        C_1 = C_coeffs[1] # This is Delta y_{n-1}
    
        y_diff = y_target - y_tmp #y_tmp = y_n
    
        sum_higher_order = 0.0
        for i in range(2, n + 1):
            C_i = C_coeffs[i] # This is Delta^i y_{n-i}
            D_i = C_i / math.factorial(i)
            B_i_t = evaluate_basis_product(t, i, condition)
            sum_higher_order += D_i * B_i_t
        
        return (y_diff - sum_higher_order) / C_1

In [56]:
def inverse_interpolation_newton_fixed_point(points, y_target, eps, condition):
    if condition == 1:
        """
        Finds the x value for a given y_target using the fixed-point iteration
        method derived from Newton's Forward formula.
    
        Args:
            points (list of tuples): List of (x, y) data points. 
                                 Must be evenly spaced and monotonic.
            y_target (float): The y-value to find the x for.
            eps (float): The tolerance for convergence |t_{k+1} - t_k| < eps.
        
        Returns:
            float: The interpolated x-value.
        """
    
        n = len(points) - 1
        if n < 1:
            raise ValueError("At least two points are required.")
    
        # --- 1. Setup and Difference Table ---
        x_0 = points[0][0]
        y_0 = points[0][1]
        h = points[1][0] - points[0][0]
    
        diff_table, C_coeffs = EvenDifference(points, condition=1)
    
        C_1 = C_coeffs[1]
        if np.isclose(C_1, 0):
            raise ValueError("Delta_y0 (C_1) is zero, this method cannot be used. "
                         "The function may not be monotonic.")
                         
        print("--- Inverse Interpolation Setup ---")
        print(f"Target y_target = {y_target}")
        print(f"Interval starts at x_0 = {x_0}, y_0 = {y_0}")
        print(f"h = {h:.2f}, C_1 (Delta_y0) = {C_1:.12f}")
    
        # --- 2. Initial Guess t_0 ---
        t_0 = (y_target - y_0) / C_1
        print(f"Initial guess: t_0 = (y_target - y_0) / C_1 = {t_0:.12f}")
        print("-" * 30)

        # --- 3. Perform Fixed-Point Iteration ---
        print("--- Iteration Steps ---")
    
        t_k = t_0
        results = []
    
        for k in range(100): # Max 100 iterations
            # t_{k+1} = phi(t_k)
            t_kplus1 = phi_function(t_k, y_target, y_0, C_coeffs, condition)
        
            error = abs(t_kplus1 - t_k)
        
            results.append({
                'k': k,
                't_k': t_k,
                't_k+1 = phi(t_k)': t_kplus1,
                'error = |t_k+1 - t_k|': error
            })
        
            t_k = t_kplus1
        
            if error < eps:
                break
            
        df_results = pd.DataFrame(results, columns=['k', 't_k', 't_k+1 = phi(t_k)', 'error = |t_k+1 - t_k|'])
    
        # --- 4. Final Result ---
        final_t = t_k
        final_x = x_0 + final_t * h
    
        print(f"\nConvergence reached in {k+1} iterations.")
        print(f"Final approximated x = x_0 + t*h = {x_0} + {final_t:.12f} * {h:.2f}")
    
        return diff_table, df_results, final_t, final_x
    
    if condition == 0:
        """
        Finds the x value for a given y_target using the fixed-point iteration
        method derived from Newton's Backward formula.
    
        Args:
            points (list of tuples): List of (x, y) data points. 
                                 Must be evenly spaced and monotonic.
            y_target (float): The y-value to find the x for.
            eps (float): The tolerance for convergence |t_{k+1} - t_k| < eps.
        
        Returns:
            float: The interpolated x-value.
        """
    
        n = len(points) - 1
        if n < 1:
            raise ValueError("At least two points are required.")
    
        # --- 1. Setup and Difference Table ---
        x_n = points[-1][0]
        y_n = points[-1][1]
        h = points[1][0] - points[0][0] # h is constant
    
        # Call EvenDifference with condition=0 to get BACKWARD coefficients
        diff_table, C_coeffs_bwd = EvenDifference(points, condition=0)
    
        C_1 = C_coeffs_bwd[1] # C_1 = Delta y_{n-1}
        if np.isclose(C_1, 0):
            raise ValueError("Delta_y_{n-1} (C_1) is zero, this method cannot be used. "
                         "The function may not be monotonic.")
                         
        print("--- Inverse Interpolation Setup (Newton-Backward) ---")
        print(f"Target y_target = {y_target}")
        print(f"Interval ends at x_n = {x_n}, y_n = {y_n}")
        print(f"h = {h:.2f}, C_1 (Delta y_n-1) = {C_1:.12f}")
    
        # --- 2. Initial Guess t_0 ---
        t_0 = (y_target - y_n) / C_1
        print(f"Initial guess: t_0 = (y_target - y_n) / C_1 = {t_0:.12f}")
        print("-" * 30)

        # --- 3. Perform Fixed-Point Iteration ---
        print("--- Iteration Steps ---")
    
        t_k = t_0
        results = []
    
        for k in range(100): # Max 100 iterations
            # t_{k+1} = phi(t_k)
            t_kplus1 = phi_function(t_k, y_target, y_n, C_coeffs_bwd, condition)
        
            error = abs(t_kplus1 - t_k)
        
            results.append({
                'k': k,
                't_k': t_k,
                't_k+1 = phi(t_k)': t_kplus1,
                'error = |t_k+1 - t_k|': error
            })
        
            t_k = t_kplus1
        
            if error < eps:
                break
            
        df_results = pd.DataFrame(results, columns=['k', 't_k', 't_k+1 = phi(t_k)', 'error = |t_k+1 - t_k|'])
    
        # --- 4. Final Result ---
        final_t = t_k
        final_x = x_n + final_t * h
    
        print(f"\nConvergence reached in {k+1} iterations.")
        print(f"Final approximated x = x_n + t*h = {x_n} + ({final_t:.12f}) * {h:.2f}")
    
        return diff_table, df_results, final_t, final_x

## Result

In [57]:
# 2. Define the problem
y_target_val = -1.43
tolerance = 1e-7

diff_table, df_results, t_solution, x_solution_fwd = inverse_interpolation_newton_fixed_point(
    points=monotonic_points_fwd,
    y_target=y_target_val,
    eps=tolerance,
    condition = 0
)

--- Inverse Interpolation Setup (Newton-Backward) ---
Target y_target = -1.43
Interval ends at x_n = 2.61, y_n = -1.3977
h = 0.11, C_1 (Delta y_n-1) = 0.115000000000
Initial guess: t_0 = (y_target - y_n) / C_1 = -0.280869565217
------------------------------
--- Iteration Steps ---

Convergence reached in 9 iterations.
Final approximated x = x_n + t*h = 2.61 + (-0.220791745173) * 0.11


In [58]:
print("--- Generated Finite Difference Table ---")

diff_table.style

--- Generated Finite Difference Table ---


,x_i,y_i,Order 1,Order 2,Order 3
0,2.265000,-1.663500,0.081600,-0.012400,0.058200
1,2.380000,-1.581900,0.069200,0.045800,nan
2,2.495000,-1.512700,0.115000,nan,nan
3,2.610000,-1.397700,nan,nan,nan


In [61]:
print("--- Iteration Steps ---")

df_results

--- Iteration Steps ---


,k,t_k,t_k+1 = phi(t_k),error = |t_k+1 - t_k|
0,0,-0.280869565217,-0.211360469753,0.069509095464
1,1,-0.211360469753,-0.222529322565,0.011168852812
2,2,-0.222529322565,-0.220479340102,0.002049982463
3,3,-0.220479340102,-0.220848182342,0.000368842240
4,4,-0.220848182342,-0.220781572895,0.000066609447
5,5,-0.220781572895,-0.220793593957,0.000012021062
6,6,-0.220793593957,-0.220791424246,0.000002169711
7,7,-0.220791424246,-0.220791815854,0.000000391608
8,8,-0.220791815854,-0.220791745173,0.000000070681


In [60]:
print(f"Final approximated t = {t_solution:.12f}")
print(f"Final approximated x = {x_solution_fwd:.12f}")

Final approximated t = -0.220791745173
Final approximated x = 2.584608949305
